<center><h1>Maltese Christian Statue (MCS) YOLO Classification</h1>
<h2>Matthias Bartolo</h2>

</center>

### Preparing the MCS dataset


The Dataset features 14 Categories of Christian Statues found in Malta specifically in the parish church of `Ħaż-Żebbuġ` dedicated to `St Philip of Agira`. Note that the images retrieved for the creation of such dataset, were extracted from public domain sources and are not intended for commercial use.

In [1]:
import os

# Defining the dataset path
dataset_path = 'MCS Dataset/'

# Retrieving the categories
categories = os.listdir(dataset_path)

# Printing the categories
for i, category in enumerate(categories):
    print('\033[35m' + str(i) + '. ' + category + '\033[0m')

0. Christmas Crib
1. Jesus has Risen
2. Jesus praying in Gethsemane
3. Our Lady of Grace
4. Saint Joseph
5. Saint Philip of Agira
6. Simon of Cyrene
7. The Betrayal of Judas
8. The Cross
9. The Crucifixion
10. The Ecce Homo
11. The Flogged
12. The Lady of Sorrows
13. The Last Supper
14. The Monument
15. The Redeemer
16. The Veronica


### Formatting the data to be used by the YOLO model
The data is split into training and testing sets. The training set is used to train the model, while the testing set is used to evaluate the model's performance. The data is split into 80% training and 20% testing.

In [2]:
do_split = False

if do_split:
    import os
    import shutil

    # Define paths
    original_dataset_path = 'Augmented MCS Dataset1k/'  # Original dataset directory
    output_base_path = 'Augmented MCS Dataset Split/'        
    train_dir = os.path.join(output_base_path, 'train')
    test_dir = os.path.join(output_base_path, 'test')

    # Create output directories
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Retrieve categories
    categories = os.listdir(original_dataset_path)

    # Process each category
    for category in categories:
        category_path = os.path.join(original_dataset_path, category)
        image_names = os.listdir(category_path)
        
        # Split the image names into train (80%) and test (20%) sets
        train_count = int(0.8 * len(image_names))
        train_images = image_names[:train_count]
        test_images = image_names[train_count:]
        
        # Create subdirectories for the category in train and test directories
        train_category_dir = os.path.join(train_dir, category)
        test_category_dir = os.path.join(test_dir, category)
        os.makedirs(train_category_dir, exist_ok=True)
        os.makedirs(test_category_dir, exist_ok=True)
        
        # Move train images
        for image_name in train_images:
            src_path = os.path.join(category_path, image_name)
            dst_path = os.path.join(train_category_dir, image_name)
            shutil.copy(src_path, dst_path)
        
        # Move test images
        for image_name in test_images:
            src_path = os.path.join(category_path, image_name)
            dst_path = os.path.join(test_category_dir, image_name)
            shutil.copy(src_path, dst_path)

    print('\033[35m' + 'Dataset successfully reorganized into CIFAR-10-style structure.' + '\033[0m')


### Training the YOLO model

In [3]:
# import YOLO model
from ultralytics import YOLO

# Load a model
model = YOLO('yolo11m-cls.pt') # load a pretrained model (recommended for training)

# Train the model
model.train(data='D:/GitHub/Maltese-Christian-Statue-Classifier/Augmented MCS Dataset Split', 
            epochs=100, 
            batch=128, 
            device= 0,  # Use GPU 0
            optimizer= 'Adam',  # Use Adam optimizer
            cache= True, #'disk',  # Cache images for faster training
            patience= 15,  # epochs to wait before decreasing LR
            weight_decay= 0.0005,  # Weight decay
            lr0= 0.001,  # Initial learning rate
            momentum= 0.9,  # Momentum
            dropout= 0.5,  # Dropout rate
            val= True,  # Run validation during training
            plots= True,  # Run plots during training
            verbose= True,  # Print detailed results
            )

New https://pypi.org/project/ultralytics/8.3.63 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.59  Python-3.9.19 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: task=classify, mode=train, model=yolo11x-cls.pt, data=D:/GitHub/Maltese-Christian-Statue-Classifier/Augmented MCS Dataset Split, epochs=100, time=None, patience=15, batch=128, imgsz=224, save=True, save_period=-1, cache=True, device=0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes

train: Scanning D:\GitHub\Maltese-Christian-Statue-Classifier\Augmented MCS Dataset Split\train... 20387 images, 0 corrupt: 100%|██████████| 20387/20387 [00:00<?, ?it/s]


WARNING  Classification `cache_ram` training has known memory leak in https://github.com/ultralytics/ultralytics/issues/9824, setting `cache_ram=False`.


val: Scanning D:\GitHub\Maltese-Christian-Statue-Classifier\Augmented MCS Dataset Split\test... 5100 images, 0 corrupt: 100%|██████████| 5100/5100 [00:00<?, ?it/s]


optimizer: Adam(lr=0.001, momentum=0.9) with parameter groups 82 weight(decay=0.0), 83 weight(decay=0.001), 83 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs\classify\train4
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

                   all       0.81      0.953
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 59, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



74 epochs completed in 3.884 hours.
Optimizer stripped from runs\classify\train4\weights\last.pt, 57.0MB
Optimizer stripped from runs\classify\train4\weights\best.pt, 57.0MB

Validating runs\classify\train4\weights\best.pt...
Ultralytics 8.3.59  Python-3.9.19 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLO11x-cls summary (fused): 227 layers, 28,354,193 parameters, 0 gradients, 110.3 GFLOPs
WARNING  Dataset 'split=val' not found, using 'split=test' instead.
train: D:\GitHub\Maltese-Christian-Statue-Classifier\Augmented MCS Dataset Split\train... found 20387 images in 17 classes  
val: None...
test: D:\GitHub\Maltese-Christian-Statue-Classifier\Augmented MCS Dataset Split\test... found 5100 images in 17 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 20/20 [00:05<00:00,  3.40it/s]


                   all      0.813      0.953
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train4


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019C98D3B850>
curves: []
curves_results: []
fitness: 0.8826470375061035
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8125489950180054, 'metrics/accuracy_top5': 0.9527450799942017, 'fitness': 0.8826470375061035}
save_dir: WindowsPath('runs/classify/train4')
speed: {'preprocess': 0.048968417971741916, 'inference': 0.652008664374258, 'loss': 0.0, 'postprocess': 0.00019620446597828586}
task: 'classify'
top1: 0.8125489950180054
top5: 0.9527450799942017

### Evaluating the YOLO model

In [4]:
# metrics = model.val(split="test") # no arguments needed, dataset and settings remembered
# metrics.top1 # top1 accuracy
# metrics.top5 # top5 accuracy

### Prediction on Images

In [5]:
from ultralytics import YOLO
import cv2
import os
from matplotlib import pyplot as plt
import torch
import numpy as np

# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Categories in English and Maltese
categories_english = {
    0: 'Christmas Crib', 
    1: 'Jesus has Risen', 
    2: 'Jesus praying in Gethsemane', 
    3: 'Our Lady of Grace', 
    4: 'Saint Joseph', 
    5: 'Saint Philip of Agira', 
    6: 'Simon of Cyrene', 
    7: 'The Betrayal of Judas', 
    8: 'The Cross', 
    9: 'The Crucifixion', 
    10: 'The Ecce Homo', 
    11: 'The Flogged', 
    12: 'The Lady of Sorrows', 
    13: 'The Last Supper', 
    14: 'The Monument', 
    15: 'The Redeemer', 
    16: 'The Veronica'
}

categories_maltese = {
    0: 'Il-Presepju', 
    1: 'L-Irxoxt', 
    2: 'Ġesù fl-Ort tal-Ġetsemani', 
    3: 'Il-Madonna tal-Grazzja', 
    4: 'San Ġużepp', 
    5: 'San Filep ta’ Aġġira', 
    6: 'Xmun min Ċireni', 
    7: 'It-Tradiment ta’ Ġuda', 
    8: 'Is-Salib', 
    9: 'Il-Vara Il-Kbira', 
    10: 'L-Ecce Homo', 
    11: 'Il-Marbut', 
    12: 'Id-Duluri', 
    13: 'L-Aħħar Ċena', 
    14: 'Il-Monument', 
    15: 'Ir-Redentur', 
    16: 'Il-Veronica'
}


def predict(image_paths, model, size=(244, 244)):
    """Predict the class of images and plot results using image paths."""
    predictions = []

    for path in image_paths:
        # Read and resize the image
        image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        resized_img = cv2.resize(image, size)
        resized_img = resized_img / 255.0  # Normalize
        resized_img = resized_img.transpose(2, 0, 1)  # Convert to (C, H, W)
        resized_img = resized_img[None, ...]  # Add batch dimension

        # Run prediction
        results = model.predict(path)
        pred_probs = results[0].probs.data.cpu().numpy()
        pred_class = np.argmax(pred_probs)
        pred_prob = round(np.max(pred_probs) * 100, 2)

        # Plot image with prediction
        plt.figure(figsize=(10, 7))
        plt.imshow(image)
        plt.title(f'Prediction: {categories_maltese[pred_class]} ({pred_prob}%)', fontsize=18, color='white', backgroundcolor='red')
        plt.axis('off')
        plt.show()

        # Save predictions
        predictions.append((path, categories_english[pred_class], pred_prob))

    return predictions

# Dataset path and model loading
dataset_path = "MCS Dataset/"
model = YOLO("models/MCS-Classifyv2.pt").to(device)

# Load subset of image paths
subset_amount = 2
image_paths = []

categories = list(categories_english.values())
for category in categories:
    category_path = os.path.join(dataset_path, category)
    image_files = os.listdir(category_path)[:subset_amount]
    image_paths.extend([os.path.join(category_path, image_name) for image_name in image_files])

# Predict classes using image paths
predictions = predict(image_paths, model)

# Print predictions
for path, predicted_class, confidence in predictions:
    print(f"Image: {os.path.basename(path)}, Predicted: {predicted_class}, Confidence: {confidence}%")



image 1/1 d:\GitHub\Maltese-Christian-Statue-Classifier\MCS Dataset\Christmas Crib\121960532_4030654523615342_425859488410538693_n.jpg: 224x224 Saint Philip of Agira 0.67, The Veronica 0.27, The Ecce Homo 0.02, The Monument 0.01, The Crucifixion 0.01, 3.5ms
Speed: 13.5ms preprocess, 3.5ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


<Figure size 1000x700 with 1 Axes>


image 1/1 d:\GitHub\Maltese-Christian-Statue-Classifier\MCS Dataset\Christmas Crib\121988132_4030650750282386_6216238816359705572_n.jpg: 224x224 The Flogged 0.79, Jesus praying in Gethsemane 0.12, The Ecce Homo 0.05, Jesus has Risen 0.03, The Crucifixion 0.01, 3.0ms
Speed: 6.6ms preprocess, 3.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


<Figure size 1000x700 with 1 Axes>


image 1/1 d:\GitHub\Maltese-Christian-Statue-Classifier\MCS Dataset\Jesus has Risen\11063760_10153526942450732_3570215296722781252_o.jpg: 224x224 Jesus has Risen 0.82, The Ecce Homo 0.08, The Monument 0.07, The Flogged 0.02, The Crucifixion 0.01, 4.0ms
Speed: 4.0ms preprocess, 4.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


...


image 1/1 d:\GitHub\Maltese-Christian-Statue-Classifier\MCS Dataset\The Veronica\10258518_592725520834843_1233021636814841544_o.jpg: 224x224 The Veronica 1.00, The Lady of Sorrows 0.00, The Flogged 0.00, The Ecce Homo 0.00, The Betrayal of Judas 0.00, 15.0ms
Speed: 12.5ms preprocess, 15.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


<Figure size 1000x700 with 1 Axes>


image 1/1 d:\GitHub\Maltese-Christian-Statue-Classifier\MCS Dataset\The Veronica\11136205_791690694271657_5226313664779022469_o.jpg: 224x224 The Veronica 1.00, The Flogged 0.00, The Lady of Sorrows 0.00, Jesus has Risen 0.00, The Ecce Homo 0.00, 16.0ms
Speed: 12.1ms preprocess, 16.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


<Figure size 1000x700 with 1 Axes>

Image: 121960532_4030654523615342_425859488410538693_n.jpg, Predicted: Saint Philip of Agira, Confidence: 67.26%
Image: 121988132_4030650750282386_6216238816359705572_n.jpg, Predicted: The Flogged, Confidence: 78.78%
Image: 11063760_10153526942450732_3570215296722781252_o.jpg, Predicted: Jesus has Risen, Confidence: 81.98%
Image: 12022442_10153526942610732_127221472302263044_o.jpg, Predicted: Jesus has Risen, Confidence: 100.0%
Image: 011.JPG, Predicted: Jesus praying in Gethsemane, Confidence: 100.0%
Image: 10007326_576843849089677_365853481_o.jpg, Predicted: Jesus praying in Gethsemane, Confidence: 100.0%
Image: 116623713_918362858663099_4260138187212353875_n.jpg, Predicted: Our Lady of Grace, Confidence: 100.0%
Image: 120194429_669271723710629_565051212040778366_n.jpg, Predicted: Our Lady of Grace, Confidence: 99.97%
Image: 104473090_2926755327378742_1890932265076098324_n.jpg, Predicted: Saint Joseph, Confidence: 99.69%
Image: 107609219_2979287158792225_8516499388614223790_n.jpg, Pr